In [ ]:
# load text
# load search terms
# make regex
# save resuls

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv
hv.extension('bokeh', 'matplotlib')

In [ ]:
# get search terms
terms_file = '/home/jvdzwaan/Downloads/Farad_Infixes (1).txt'
with open(terms_file, encoding='utf-8') as f:
    lines = f.readlines()

terms = [t.strip() for t in lines]
len(terms)

In [ ]:
# load text
text_file = '/home/jvdzwaan/code/fiqh_corpus/0483IbnAhmadSarakhsi.Mabsut.txt'
with open(text_file, encoding='utf-8') as f:
    text = f.read()
print(len(text))

In [ ]:
print(terms[0])

In [ ]:
import re

#regex = r'(.+\s){0,5}.*فرض.*(.+\s){0,5}'

for match in re.finditer(r'\S'+terms[0]+'\S', text):
    #print(match.span())
    b, e = match.span()
    if b < 25:
        b = 0
    else:
        b = b-25
    if e > len(text) - 25:
        e = len(text)
    else:
        e = e + 27
    print(text[b:e])
    print('---')

In [ ]:
from tqdm import tqdm

results = []

for t in terms:
    regexes = [r'\S'+t+'\S', r'\s'+t+'\S', r'\S'+t+'\s']
    for regex in regexes:
        print(regex)
        for match in re.finditer(r'\S'+terms[0]+'\S', text):
            b, e = match.span()
            if b < 25:
                b = 0
            else:
                b = b-25
            if e > len(text) - 25:
                e = len(text)
            else:
                e = e + 27
            context = text[b:e]
            results.append({'term': t, 'context': context})

In [ ]:
import pandas as pd

df = pd.DataFrame(results)
df

In [ ]:
df.to_csv('0483IbnAhmadSarakhsi.Mabsut-farad_infix_matches.csv', encoding='utf-8')

In [ ]:
from nlppln.utils import get_files

in_dir = '/home/jvdzwaan/data/adh-corpora/fiqh_corpus/txt/'
corpus = get_files(in_dir)
print(len(corpus))

In [ ]:
%%time

import re

import pandas as pd
from tqdm import tqdm_notebook as tqdm

from nlppln.utils import remove_ext, get_files

from adhtools.utils import analyzer_xml2df

def text_regex(fname, term):
    with open(fname) as f:
        text = f.read()
        
    num_matches1 = 0
    for match in re.finditer(r'\b.?'+term+r'.?\b', text):
        num_matches1 += 1
    
    num_matches2 = 0
    for match in re.finditer(term, text):
        num_matches2 += 1
    
    return num_matches1, num_matches2

def text_results(in_files, term):
    res = []
    for in_file in tqdm(in_files):
        res.append(text_regex(in_file, term))
    return res

def xml_regex(fname, term):
    #print(fname)
    data = analyzer_xml2df(fname)
    #print(data)
    words = data['word']
    text = ' '.join(words)
    
    #print(len(text))
    
    regex = re.compile(r'\b.?'+term+r'.?\b')
    
    num_matches1 = 0
    for match in re.finditer(regex, text):
        num_matches1 += 1
        
    num_matches2 = 0
    for match in re.finditer(term, text):
        num_matches2 += 1

    return num_matches1, num_matches2

def xml_results(in_files, term):
    res = []
    for in_file in tqdm(in_files):
        res.append(xml_regex(in_file, term))
    return res

def count_matches(in_dir_text, in_dir_xml, term):
    result = pd.DataFrame()
    in_files = get_files(in_dir_text)
    text_res = text_results(in_files, term) 
    result['text'] = [a for a, b in text_res]
    result['text2'] = [b for a, b in text_res]
    
    in_files = get_files(in_dir_xml)
    xml_res = xml_results(in_files, term) 
    result['xml'] = [a for a, b in xml_res]
    result['xml2'] = [b for a, b in xml_res]
    
    index = [remove_ext(fname) for fname in in_files]
    result['idx'] = index
    result = result.set_index('idx')
    
    return result

In [ ]:
%%time
data2 = count_matches('/home/jvdzwaan/data/adh-corpora/fiqh_corpus/txt/', 
                      '/home/jvdzwaan/data/tmp/adh/2018-11-07-fiqh-alkhalil/', 
                      'مصلح')
data2.head()

In [ ]:
data1.head()

In [ ]:
data2.head()

In [ ]:
data2.plot(kind='bar', figsize=(15,8))

In [ ]:
data2[['text', 'text2']].plot(figsize=(15,8))

In [ ]:
data2['text2'] == data2['xml2']

In [ ]:
data2.sum()

In [ ]:
data2.loc['1266MuhammadHasanNajafiJawhari.JawahirKalam']

In [ ]:
# create query by inserting keyword
# get url
# check status code
# read xml

In [ ]:
import requests

term = 'مصلح'
url = 'http://localhost:8080/blacklab-server/Fiqh/hits'
params = {'patt': '".*{}.*"'.format(term), 'number': 100, 'group': 'field:BookURI'}

r = requests.get(url, params=params)
print(r.url)
print(r.status_code)
#print(r.text)

In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(r.text, 'xml')
#print(soup)

In [ ]:
noh = soup.find('numberOfHits')
print(noh.text)

In [ ]:
hgs = soup.find_all('hitgroup')
result = {}
for hg in hgs:
    #print(hg)
    name = hg.identityDisplay.text
    value = int(hg.size.text)
    result[name] = value
#result

In [ ]:
pd.Series(result)

In [ ]:
data2['blacklab'] = pd.Series(result)

In [ ]:
data2['blacklab'] == data2['xml2']

In [ ]:
data2[data2['blacklab'] != data2['xml2']]

In [ ]:
data2[['blacklab', 'xml2']].plot(figsize=(15,8), kind='bar')

In [ ]:
in_dir_xml = '/home/jvdzwaan/data/tmp/adh/2018-11-07-fiqh-alkhalil/'
in_files = get_files(in_dir_xml)
index = [remove_ext(fname) for fname in in_files]
print(index)
data1['idx'] = index
data1 = data1.set_index('idx')

In [ ]:
data1

In [ ]:
%%time
#xml_files = get_files('/home/jvdzwaan/data/tmp/adh/2018-11-07-fiqh-alkhalil')
#for in_file in xml_files:
#    print(in_file)

def xml_regex(fname, term):
    #print(fname)
    data = analyzer_xml2df(fname)
    #print(data)
    words = data['word']
    text = ' '.join(words)
    
    print(len(text))
    
    regex = re.compile(r'\b.?'+term+r'.?\b')
    
    num_matches1 = 0
    for match in re.finditer(regex, text):
        num_matches1 += 1
        
    num_matches2 = 0
    for word in words:
        if re.search(term, word):
            num_matches2 += 1

    return num_matches1, num_matches2

r = xml_regex('/home/jvdzwaan/data/tmp/adh/2018-11-07-fiqh-alkhalil/0179MalikIbnAnas.Muwatta.xml', 'مصالح')

In [ ]:
r

In [ ]:
print(xml_files[0])

In [ ]:
%%time
r = analyzer_xml2df('/home/jvdzwaan/data/tmp/adh/2018-11-07-fiqh-alkhalil/0179MalikIbnAnas.Muwatta.xml')

In [ ]:
words = r['word']
text = ' '.join(words)

In [ ]:
%%time
term = 'مصلح'
num_matches = 0
for match in re.finditer(r'\b.*'+term+r'.*\b', text):
    num_matches += 1

In [ ]:
num_matches

In [ ]:
num_matches = 0
for word in words:
    if re.search(term, word):
        num_matches += 1
print(num_matches)

In [ ]:
for in_file in corpus:
    text_regex(in_file, 'مصالح')

In [ ]:
for in_file in corpus:
    text_regex(in_file, 'قال')